## RentItNow car sharing

You are software engineer at **RentItNow**, a car sharing company located in SimpleTown.
SimpleTown is a rounded village dived in three concentric circles: **Inner Circle**, **Middle Circle**, **Outer Circle**.

Your boss asks you to develop a new software to manage company's cars and users.
The boss needs to:
- Add, update and remove cars;
- Check the status of the car: location, total distance traveled, next service time, availability.
- Add, update and remove users;

A car has (at least)
- a type, 
- a license plate, 
- a brand,
- a name

There are three types of car: 
- **ECO**, max 2 persons,
- **MID-CLASS**, max 4 persons,
- **DELUXE**, max 7 persons.

Each type of car has a rental price per km:
- **ECO**: 1$/km;
- **MID-CLASS**: 2$/km;
- **DELUXE**: 5$/km

Each type of car has a fixed speed:
- **ECO**: 15km/h
- **MID-CLASS**: 25km/h
- **DELUXE**: 50km/h

The company must **service** its cars every 1500km. Service takes 1 day and cost 300$. Car cannot be rented on that day.

A **user** can register to the company service, update its data and delete its account. User has (at least):
- name, 
- surname, 
- address, 
- credit card,
- driving license

A user can ask for a car: select type of car, number of passengers, starting circle and destination circle. 

Travel distance is computed based on **hops**; an hop is 5km; an hops is going from one circle to the next one. **Always counts 1 hop for the starting circle**
(e.g. travelling from Inner Circle to Middle Circle is 2 hops, not 1 hop). Travelling in the same circle is 1 hop.

The software select the best car for the user based on some metric (up to you), calculate the cost of the trip and make the payment.
If no car is available, the software presents the user an expected waiting time. 


In [137]:
class Circle:
    def __init__(self, name):
        self.name = name

    def distance_to(self, other_circle):
        if self.name == other_circle.name:
            return 5  # 1 hop
        elif self.name == "Inner Circle" and other_circle.name == "Middle Circle":
            return 10  # 2 hops
        elif self.name == "Middle Circle" and other_circle.name == "Outer Circle":
            return 10  # 2 hops
        elif self.name == "Inner Circle" and other_circle.name == "Outer Circle":
            return 15  # 3 hops
        else:
            raise ValueError(f"Invalid circles for calculating distance: {self.name} and {other_circle.name}")

In [133]:
class Car:
    TYPE_PRICES = {
        "ECO": 1,
        "MID-CLASS": 2,
        "DELUXE": 5,
    }

    TYPE_SPEEDS = {
        "ECO": 15,
        "MID-CLASS": 25,
        "DELUXE": 50,
    }

    def __init__(self, car_type, license_plate, brand, name):
        self.type = car_type
        self.license_plate = license_plate
        self.brand = brand
        self.name = name
        self.total_distance = 0
        self.next_service_distance = 1500
        self.availability = True
        self.travel_time = 0
    
    def calculate_cost(self, distance):
        return distance * self.TYPE_PRICES[self.type]

    def calculate_travel_time(self, distance):
        self.travel_time = distance / self.TYPE_SPEEDS[self.type]
        return distance / self.TYPE_SPEEDS[self.type]
        
    def get_travel_time(self):
        return self.travel_time
        
    def service(self, rent_it_now: RentItNow):
        self.next_service_distance = 1500
        self.availability = False
        rent_it_now.update_bank_account(-300)
        print(f"service done for {self.license_plate}")
        

    def reserve(self):
        self.availability = False
        
    def update_availability(self):
        if self.next_service_time and self.next_service_time < datetime.now():
            self.availability = True

    def set_total_distance(self, distance, rent_it_now: RentItNow):
        self.total_distance += distance
        if self.total_distance >= 1500:
            self.service(rent_it_now)
        
    def print_car_info(self):
        print(self.license_plate)
        print(f"Total Distance Traveled: {self.total_distance} km")
        print(f"Next Service in: {1500 - self.total_distance} km")
        print(f"Availability: {'Available' if self.availability else 'Not Available'}")

In [120]:
import datetime
from typing import List, Optional

class User:
    def __init__(self, name, surname, address, credit_card, driving_license, selected_car_type: str, num_passengers: int, 
                 start_circle: str, destination_circle: str):
        self.name = name
        self.surname = surname
        self.address = address
        self.credit_card = credit_card
        self.driving_license = driving_license
        self.reservations = []
        self.selected_car_type = selected_car_type
        self.num_passengers = num_passengers
        self.start_circle = Circle(start_circle)
        self.destination_circle = Circle(destination_circle)
        
    def reserve_car(self, rent_it_now: 'RentItNow'):
        best_car = rent_it_now.find_best_car(self.selected_car_type, self.num_passengers, self.start_circle, self.destination_circle)
        
        if not best_car:
            waiting_time = rent_it_now.get_last_saved_car().get_travel_time()
            return f"No cars are available at the moment. The expected waiting time is {waiting_time} hours."
        
        total_distance = self.start_circle.distance_to(self.destination_circle)
        best_car.set_total_distance(total_distance, rent_it_now)
        travel_time = best_car.calculate_travel_time(total_distance)
        cost = best_car.calculate_cost(total_distance)

        best_car.reserve()
        self.reservations.append((best_car, self.start_circle, self.destination_circle, datetime.timedelta(minutes=travel_time), cost))

        rent_it_now.update_bank_account(cost)
        
        return f"Car {best_car.license_plate} has been reserved. The travel time is {travel_time} hours, and the cost is ${cost}."

    def print_user_info(self):
        print("Name:", self.name)
        print("Surname:", self.surname)
        print("Address:", self.address)
        print("Credit Card:", self.credit_card)
        print("Driving License:", self.driving_license)
        print("Selected Car Type:", self.selected_car_type)
        print("Number of Passengers:", self.num_passengers)

In [111]:
class RentItNow:
    def __init__(self):
        self.cars = []
        self.users = []
        self.circles = {
            "Inner Circle": Circle("Inner Circle"),
            "Middle Circle": Circle("Middle Circle"),
            "Outer Circle": Circle("Outer Circle"),
        }
        self.bank_account = 0
        
    def iterate_cars(self):
        for car in self.cars:
            yield car
            
    def add_car(self, car):
        self.cars.append(car)

    def update_car(self, car):
        for idx, c in enumerate(self.cars):
            if c.license_plate == car.license_plate:
                self.cars[idx] = car
                break
    
    def remove_car(self, license_plate):
        self.cars = [car for car in self.cars if car.license_plate != license_plate]
        
    def get_last_saved_car(self):
        if self.cars:
            return self.cars[-1] 
        else:
            return None
    
    def iterate_users(self):
        for user in self.users:
            yield user
            
    def add_user(self, user):
        self.users.append(user)

    def update_user(self, user):
        for idx, u in enumerate(self.users):
            if u.name == user.name and u.surname == user.surname:
                self.users[idx] = user
                break

    def remove_user(self, name, surname):
        self.users = [user for user in self.users if user.name != name or user.surname != surname]

    def update_bank_account(self, amount):
        self.bank_account += amount

    def get_bank_account(self):
        return self.bank_account
        
    #def find_available_car(self, car_type, num_passengers, start_circle, destination_circle):
        # Implement the logic to find the best available car based on your chosen metric.
        #pass

    def find_best_car(self, car_type: str, num_passengers: int, start_circle, destination_circle) -> Optional['Car']:
        available_cars = [car for car in self.cars if car.availability]

        if not available_cars:
            return None

        best_car = min(available_cars, key=lambda car: car.calculate_cost(start_circle.distance_to(destination_circle)))

        return best_car

In [141]:
def main():
    # Initialize the RentItNow system
    rent_it_now = RentItNow()

    # Add some cars to the system
    eco_car = Car("ECO", "ECO123", "Tesla", "Model S")
    mid_class_car = Car("MID-CLASS", "MID456", "Toyota", "Camry")
    deluxe_car = Car("DELUXE", "DEL789", "Mercedes-Benz", "S-Class")
    suv_car = Car("MID-CLASS", "SUV101", "Ford", "Explorer")
    compact_car = Car("ECO", "CMP202", "Honda", "Civic")
    luxury_car = Car("DELUXE", "LUX303", "BMW", "7 Series")
    
    rent_it_now.add_car(eco_car)
    rent_it_now.add_car(mid_class_car)
    rent_it_now.add_car(deluxe_car)
    rent_it_now.add_car(suv_car)
    rent_it_now.add_car(compact_car)
    rent_it_now.add_car(luxury_car)
    
    # Add some users to the system
    user1 = User("John", "Doe", "123 Main St", "4123-4567-8901-2345", "DL123456", "ECO", 2, "Inner Circle", "Outer Circle")
    user2 = User("Jane", "Doe", "456 Elm St", "1234-5678-9012-3456", "DL654321", "MID-CLASS", 4, "Middle Circle", "Outer Circle" )
    user3 = User("Alice", "Smith", "789 Oak St", "9876-5432-1098-7654", "DL987654", "DELUXE", 6, "Inner Circle", "Inner Circle")
    user4 = User("Emily", "Johnson", "567 Pine St", "5678-9012-3456-7890", "DL135792", "ECO", 1, "Middle Circle", "Outer Circle" )
    #user5 = User("Michael", "Williams", "890 Cedar St", "7896-3452-9018-7456", "DL246813", "MID-CLAS", 3, "Inner Circle", "Outer Circle")
    #user6 = User("Sophia", "Brown", "123 Oak St", "3214-8765-9102-6543", "DL975318", "DELUXE", 7, "Middle Circle", "Middle Circle")
    
    rent_it_now.add_user(user1)
    rent_it_now.add_user(user2)
    rent_it_now.add_user(user3)
    rent_it_now.add_user(user4)
    #rent_it_now.add_user(user5)
    #rent_it_now.add_user(user6)

    print(user1.reserve_car(rent_it_now))

    print(user2.reserve_car(rent_it_now))

    print(user3.reserve_car(rent_it_now))

    print(user4.reserve_car(rent_it_now))

   # print(user5.reserve_car(rent_it_now))

    #print(user6.reserve_car(rent_it_now))

    for car in rent_it_now.iterate_cars():
        car.print_car_info()

    print(f"bank account balance:  {rent_it_now.get_bank_account()} " )
    #for user in rent_it_now.iterate_users():
     #   user.print_user_info()
    
if __name__ == "__main__":
    main()

Car ECO123 has been reserved. The travel time is 1.0 hours, and the cost is $15.
Car CMP202 has been reserved. The travel time is 0.6666666666666666 hours, and the cost is $10.
Car MID456 has been reserved. The travel time is 0.2 hours, and the cost is $10.
Car SUV101 has been reserved. The travel time is 0.4 hours, and the cost is $20.
ECO123
Total Distance Traveled: 15 km
Next Service in: 1485 km
Availability: Not Available
MID456
Total Distance Traveled: 5 km
Next Service in: 1495 km
Availability: Not Available
DEL789
Total Distance Traveled: 0 km
Next Service in: 1500 km
Availability: Available
SUV101
Total Distance Traveled: 10 km
Next Service in: 1490 km
Availability: Not Available
CMP202
Total Distance Traveled: 10 km
Next Service in: 1490 km
Availability: Not Available
LUX303
Total Distance Traveled: 0 km
Next Service in: 1500 km
Availability: Available
bank account balance:  55 
